In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=Turnovers&agg=PerGame"
years = list(range(2001, 2022))

In [3]:
for year in years:
    with open("turnovers_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [4]:
dfs = []
for year in years:
    with open("turnovers_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    turnovers_data = pd.read_html(str(player_table))[0]
    turnovers_data["Year"] = year
    dfs.append(turnovers_data)

In [5]:
turnovers_data = pd.concat(dfs)
turnovers_data

,Rank,Player,Team,GamesG,TurnoversTO,Ave.,Per 40 min.40m,Year
0,1,"HAWKINS, MICHAELHAWKINS, M.",WKSIdea SlaskIdea Slask,14,52,3.71,4.65,2001
1,2,"TURKCAN, MIRSADTURKCAN, M.",CSKCSKA MoscowCSKA Moscow,17,62,3.65,4.15,2001
2,3,"EPPEHEIMER, BRETTEPPEHEIMER, B.",LONKinder London TowersKinder London Towers,12,42,3.50,4.30,2001
3,3,"ROBINSON, RUMEALROBINSON, R.","2 ZADZadarZadar,",12,42,3.50,5.01,2001
4,5,"FORD, ALPHONSOFORD, A.",OLYOlympiacosOlympiacos,20,68,3.40,3.86,2001
...,...,...,...,...,...,...,...,...
40,40,"LUNDBERG, IFFELUNDBERG, I.",CSKCSKA MoscowCSKA Moscow,24,38,1.58,3.00,2021
41,42,"LAPROVITTOLA, NICOLASLAPROVITTOLA, N.",BARFC BarcelonaFC Barcelona,38,59,1.55,3.04,2021
42,43,"WOLTERS, NATEWOLTERS, N.",CZVCrvena Zvezda mts BelgradeCrvena Zvezda mts...,31,48,1.55,2.61,2021
43,44,"SMITH, JALEENSMITH, J.",BERALBA BerlinALBA Berlin,33,51,1.55,2.94,2021


In [6]:
engine = create_engine('sqlite:///turnovers_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
turnovers_data.to_sql(
    name = "turnovers_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

979

In [9]:
turnovers_leaders_select = session.execute('select * from "turnovers_leaders"')
turnovers_leaders_select.all()

[(0, 1, 'HAWKINS, MICHAELHAWKINS, M.', 'WKSIdea SlaskIdea Slask', 14, 52, 3.71, 4.65, 2001),
 (1, 2, 'TURKCAN, MIRSADTURKCAN, M.', 'CSKCSKA MoscowCSKA Moscow', 17, 62, 3.65, 4.15, 2001),
 (2, 3, 'EPPEHEIMER, BRETTEPPEHEIMER, B.', 'LONKinder London TowersKinder London Towers', 12, 42, 3.5, 4.3, 2001),
 (3, 3, 'ROBINSON, RUMEALROBINSON, R.', '2 ZADZadarZadar,', 12, 42, 3.5, 5.01, 2001),
 (4, 5, 'FORD, ALPHONSOFORD, A.', 'OLYOlympiacosOlympiacos', 20, 68, 3.4, 3.86, 2001),
 (5, 6, 'STANOJEVIC, JOVOSTANOJEVIC, J.', 'PARPartizanPartizan', 14, 46, 3.29, 4.23, 2001),
 (6, 7, 'ALEXIS, WENDELLALEXIS, W.', 'BERAlba BerlinAlba Berlin', 14, 43, 3.07, 4.0, 2001),
 (7, 8, 'EDNEY, TYUSEDNEY, T.', 'TREBenetton BasketBenetton Basket', 19, 57, 3.0, 3.96, 2001),
 (8, 8, 'RAKOCEVIC, IGORRAKOCEVIC, I.', 'BUDBuducnostBuducnost', 14, 42, 3.0, 3.89, 2001),
 (9, 8, 'PHELPS, DERRICKPHELPS, D.', 'BERAlba BerlinAlba Berlin', 13, 39, 3.0, 3.7, 2001),
 (10, 11, 'JASIKEVICIUS, SARUNASJASIKEVICIUS, S.', 'FCBFC Barcel